## :star: GenET database module

GenET의 database module은 NCBI의 FTP 서버에서 원하는 파일들을 찾고, 손쉽게 다운로드 받을 수 있는 함수들을 제공합니다. 대표적인 기능들은 아래의 class들로 구현되어 있습니다. 
- GetGenome
- GetChromosome
- GetGene (아직 제작 중)
- GetClinVar (제작 예정)

아래의 예시 코드를 이용해서 원하는 기능들을 사용해보세요!

### 💾 Install GenET
---
우선 아래의 기능들을 사용하기 위해서, 현재 environment에 `genet`을 설치합니다. 

터미널에 아래의 명령어로 설치할 수 있습니다.

```bash
$ conda create -n genet python=3.9
$ conda activate genet
$ pip install genet
```

### 📚 NCBI
---
class `NCBI`는 NCBI의 FTP 서버와 연결하고, meta data를 local로 다운로드 받아서 원하는 데이터에 대한 각종 정보를 바로 가져올 수 있게 해줍니다. NCBI database는 수시로 업데이트되고 있기 때문에, metadata도 종종 업데이트해주는 것이 좋습니다. 

genet을 설치한 후 처음 NCBI를 불러오면, genet에서 자동으로 local environment에 metadata를 다운로드합니다. 파일의 크기가 좀 크기 때문에 다운로드에 약간 시간이 걸리지만, 한번 다운로드 받은 후에는 빠르게 정보를 가져올 수 있다는 장점이 있습니다. 

ncbi.meta를 불러오면, 현재 local environment에 다운로드 되어있는 metadata를 DataFrame 형태로 볼 수 있습니다. 

In [1]:
from genet.database import NCBI

ncbi = NCBI()

ncbi.meta.head()

,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id
0,GCF_000006515.1,PRJNA32283,SAMN02953683,AAZY00000000.2,representative genome,441375,5808,Cryptosporidium muris RN66,strain=RN66,na,...,0,84,97,TIGR,Annotation submitted by TIGR,2023/12/06,3976,3929,43,na
1,GCF_000006605.1,PRJNA224116,SAMEA3283089,na,representative genome,306537,38289,Corynebacterium jeikeium K411,strain=K411 = NCTC 11915,na,...,2,2,2,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (P...,2024/02/17,2186,2063,62,15968079
2,GCF_000006805.1,PRJNA224116,SAMN02604216,na,na,64091,2242,Halobacterium salinarum NRC-1,strain=NRC-1; ATCC 700922,na,...,3,3,3,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (P...,2023/03/24,2750,2651,52,1864501;2050644;3448465;1398080;2552415;800258...
3,GCF_000006905.1,PRJNA224116,SAMN02604005,na,na,190650,155892,Caulobacter vibrioides CB15,strain=CB15,na,...,1,1,1,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (P...,2023/02/22,3996,3829,61,11259647;16829582
4,GCF_000007005.1,PRJNA224116,SAMN02603227,na,na,273057,2287,Saccharolobus solfataricus P2,strain=P2,na,...,1,1,1,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (P...,2023/03/30,3208,2876,52,na


Meta data에서는 각 assembly에 대한 정보를 찾아볼 수 있다. 예를 들어, 'Homo sapiens'에 대해서 찾아보자.

In [3]:
df_homo_sapiens = ncbi.meta[ncbi.meta['organism_name'] == 'Homo sapiens']
df_homo_sapiens

,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id
244857,GCF_000001405.40,PRJNA168,na,na,reference genome,9606,9606,Homo sapiens,na,na,...,24,470,35611,NCBI RefSeq,GCF_000001405.40-RS_2023_10,2023/10/02,59652,20080,22158,10508508;11780052;7219534;10830953;15496913;12...
245959,GCF_009914755.1,PRJNA807723,SAMN03255769,na,na,9606,9606,Homo sapiens,na,na,...,24,24,24,NCBI RefSeq,GCF_009914755.1-RS_2023_10,2023/10/02,58360,20077,20992,35357919


2024.02.29 기준으로, NCBI FTP 서버에 등록된 'Homo sapiens'의 assembly 정보는 총 2개가 발견된다. 

- GCF_000001405.40
- GCF_009914755.1

이 중, GCF_000001405.40는 현재 연구에서 활발하게 사용되고 있는 GRCh38.p14 (asm_name)이다. 'refseq_category'을 보면, 이 GCF_000001405.40가 Homo sapiens의 reference genome으로 지정되어 있음을 볼 수 있다. 반면, GCF_009914755.1은 'refseq_category'에 na라고 되어있다. 

GenET에서는 만약 이렇게 같은 species에서 여러개의 assembly가 등록되어 있다면, reference genome으로 지정된 것을 우선으로 찾아서 사용한다. 따라서, 아래의 `GetGenome`에서 'Homo sapiens'를 입력하면 GCF_000001405.40 정보를 불러온다.

### 🧬 GetGenome
---
`GetGenome`은 특정 spacies의 genome assembly를 가져와서 그 안의 유전 서열이나 annotation 정보를 가져옵니다. `GetGenome`은 `genet.database`에서 불러올 수 있습니다. 

In [2]:
from genet.database import GetGenome

# GetGenome을 불러올 때, 
genome = GetGenome('Homo sapiens')

genome()

#assembly_accession                                           GCF_000001405.40
bioproject                                                            PRJNA168
biosample                                                                   na
wgs_master                                                                  na
refseq_category                                               reference genome
taxid                                                                     9606
species_taxid                                                             9606
organism_name                                                     Homo sapiens
infraspecific_name                                                          na
isolate                                                                     na
version_status                                                          latest
assembly_level                                                      Chromosome
release_type                                        

In [3]:
list_contents = genome.contents()
list_contents

['README.txt',
 'Annotation_comparison',
 'GCF_000001405.40_GRCh38.p14_assembly_structure',
 'GCF_000001405.40-RS_2023_10_annotation_report.xml',
 'annotation_hashes.txt',
 'RefSeq_transcripts_alignments',
 'GCF_000001405.40_GRCh38.p14_assembly_regions.txt',
 'GCF_000001405.40_GRCh38.p14_assembly_report.txt',
 'GCF_000001405.40_GRCh38.p14_assembly_stats.txt',
 'GCF_000001405.40_GRCh38.p14_cds_from_genomic.fna.gz',
 'GCF_000001405.40_GRCh38.p14_feature_count.txt.gz',
 'GCF_000001405.40_GRCh38.p14_feature_table.txt.gz',
 'GCF_000001405.40_GRCh38.p14_genomic.fna.gz',
 'GCF_000001405.40_GRCh38.p14_genomic.gbff.gz',
 'GCF_000001405.40_GRCh38.p14_genomic.gff.gz',
 'GCF_000001405.40_GRCh38.p14_genomic.gtf.gz',
 'GCF_000001405.40_GRCh38.p14_genomic_gaps.txt.gz',
 'GCF_000001405.40_GRCh38.p14_protein.faa.gz',
 'GCF_000001405.40_GRCh38.p14_protein.gpff.gz',
 'GCF_000001405.40_GRCh38.p14_pseudo_without_product.fna.gz',
 'GCF_000001405.40_GRCh38.p14_rm.out.gz',
 'GCF_000001405.40_GRCh38.p14_rm.run

위 파일들 중에서 'README.txt'를 다운로드 해보자.

In [4]:
genome.download('README.txt')

[Info] Downloading README.txt: 100%|██████████| 54.0k/54.0k [00:00<00:00, 235kB/s] 

[Info] File downloaded successfully: .//README.txt


또는 아래와 같이 download 받을 경로를 직접 지정해줄 수 있다. 아래의 예시에서는 'test_datasets' 라는 디렉토리에 저장했다. 

In [3]:
genome.download('GCF_000001405.40_GRCh38.p14_genomic.gbff.gz', download_path='test_datasets')

[Info] File downloaded successfully: test_datasets/GCF_000001405.40_GRCh38.p14_genomic.gbff.gz


### 🧬 DataFrame Converter
---
다음은 GetGenome 에서 다운로드 받은 다양한 데이터 파일을 DataFrame 형태로 변환하고, 분석하기 위한 함수들이다.

In [ ]:
from genet.database import DFConverter

gbk_filename = 'test_datasets/GCF_000001405.40_GRCh38.p14_genomic.gff.gz'


converter = DFConverter()
df_gff = converter.convert(gbk_filename)

df_gff

,seqid,source,type,start,end,score,strand,phase,attributes
0,NC_000001.11,RefSeq,region,1,248956422,.,+,.,ID=NC_000001.11:1..248956422;Dbxref=taxon:9606...
1,NC_000001.11,BestRefSeq,pseudogene,11874,14409,.,+,.,"ID=gene-DDX11L1;Dbxref=GeneID:100287102,HGNC:H..."
2,NC_000001.11,BestRefSeq,transcript,11874,14409,.,+,.,ID=rna-NR_046018.2;Parent=gene-DDX11L1;Dbxref=...
3,NC_000001.11,BestRefSeq,exon,11874,12227,.,+,.,ID=exon-NR_046018.2-1;Parent=rna-NR_046018.2;D...
4,NC_000001.11,BestRefSeq,exon,12613,12721,.,+,.,ID=exon-NR_046018.2-2;Parent=rna-NR_046018.2;D...
...,...,...,...,...,...,...,...,...,...
4900130,NC_012920.1,RefSeq,exon,15888,15953,.,+,.,ID=exon-TRNT-1;Parent=rna-TRNT;Dbxref=GeneID:4...
4900131,NC_012920.1,RefSeq,gene,15956,16023,.,-,.,"ID=gene-TRNP;Dbxref=GeneID:4571,HGNC:HGNC:7494..."
4900132,NC_012920.1,RefSeq,tRNA,15956,16023,.,-,.,ID=rna-TRNP;Parent=gene-TRNP;Dbxref=GeneID:457...
4900133,NC_012920.1,RefSeq,exon,15956,16023,.,-,.,ID=exon-TRNP-1;Parent=rna-TRNP;Dbxref=GeneID:4...


### 🧬 GetChromosome
---


In [4]:
from genet.database import GetChromosome

chromosome = GetChromosome('Homo sapiens')
chromosome()

#assembly_accession                                           GCF_000001405.40
bioproject                                                            PRJNA168
biosample                                                                   na
wgs_master                                                                  na
refseq_category                                               reference genome
taxid                                                                     9606
species_taxid                                                             9606
organism_name                                                     Homo sapiens
infraspecific_name                                                          na
isolate                                                                     na
version_status                                                          latest
assembly_level                                                      Chromosome
release_type                                        

In [2]:
# NCBI FTP 서버에서 chromosome 파일을 다운로드 할 경로를 보여줍니다.
chromosome.info.ftp_path

'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_assembly_structure/Primary_Assembly/assembled_chromosomes/FASTA/'

In [5]:
# 현재 다운로드 받을 수 있는 chromosome files의 정보를 list로 보여줍니다.
chromosome.contents()

['chr1.fna.gz',
 'chr10.fna.gz',
 'chr11.fna.gz',
 'chr12.fna.gz',
 'chr13.fna.gz',
 'chr14.fna.gz',
 'chr15.fna.gz',
 'chr16.fna.gz',
 'chr17.fna.gz',
 'chr18.fna.gz',
 'chr19.fna.gz',
 'chr2.fna.gz',
 'chr20.fna.gz',
 'chr21.fna.gz',
 'chr22.fna.gz',
 'chr3.fna.gz',
 'chr4.fna.gz',
 'chr5.fna.gz',
 'chr6.fna.gz',
 'chr7.fna.gz',
 'chr8.fna.gz',
 'chr9.fna.gz',
 'chrX.fna.gz',
 'chrY.fna.gz']

In [4]:
# 현재 경로에 chromosome 1의 FASTA 파일을 다운로드하고, 동시에 gzip 압축을 해제합니다.
chromosome.download('chr1.fna.gz', decompress=True)

[Info] Downloading chr1.fna.gz: 100%|██████████| 62.4M/62.4M [01:15<00:00, 865kB/s] 


[Info] File downloaded successfully: .//chr1.fna.gz
[Info] Decompressing gzipped file: chr1.fna.gz
